# Neon CRM SDK - Donations Management Examples

This notebook demonstrates comprehensive donation management using the Neon CRM SDK, including:
- Searching for donations with various criteria
- Creating new donations
- Updating existing donations
- Analyzing donation data
- Working with recurring donations

Donations are financial contributions from accounts and are central to most CRM workflows.

In [1]:
import json
import os
from datetime import datetime, timedelta
from typing import Dict, Any, List

from neon_crm import NeonClient

# Initialize client
client = NeonClient(
    org_id=os.getenv("NEON_ORG_ID"),
    api_key=os.getenv("NEON_API_KEY"),
    environment="production"
)

print("Neon CRM client initialized for donations management!")

Neon CRM client initialized for donations management!


## 1. Basic Donation Search

Let's start with simple donation queries to understand the data structure.

In [2]:
# Get recent donations
search_request = {
    "searchFields": [
        {
            "field": "Donation Date",
            "operator": "GREATER_THAN",
            "value": "2024-01-01"
        }
    ],
    "outputFields": [
        "Donation ID",
        "Donation Amount",
        "Donation Date",
        "Account ID",
        "Campaign Name",
        "Fund",
    ],
    "pagination": {
        "currentPage": 0,
        "pageSize": 200
    }
}

try:
    donations = list(client.donations.search(search_request))
    
    print(f"Found {len(donations)} recent donations:")
    
    total_amount = 0
    for donation in donations:
        amount = float(donation.get('Donation Amount', 0))
        date = donation.get('Donation Date', 'Unknown')
        campaign = donation.get('Campaign Name', 'No campaign')
        
        print(f"  • ${amount:,.2f} on {date} - {campaign} ")
        total_amount += amount
    
    print(f"\nTotal amount: ${total_amount:,.2f}")
    
except Exception as e:
    print(f"Error searching donations: {e}")

Found 1983 recent donations:
  • $1,000.00 on 2024-09-29 - General Campaign 
  • $560.00 on 2024-10-09 - General Campaign 
  • $560.00 on 2024-08-03 - Office Campaign 
  • $9.34 on 2024-09-11 - General Campaign 
  • $3.19 on 2024-12-18 - General Campaign 
  • $4.36 on 2024-09-18 - General Campaign 
  • $1.59 on 2024-10-02 - General Campaign 
  • $2.78 on 2024-10-09 - General Campaign 
  • $2.58 on 2024-10-17 - General Campaign 
  • $34.37 on 2024-10-23 - General Campaign 
  • $1.59 on 2024-10-30 - General Campaign 
  • $3.38 on 2024-11-03 - General Campaign 
  • $0.80 on 2024-11-10 - General Campaign 
  • $2.79 on 2024-11-20 - General Campaign 
  • $0.80 on 2024-11-24 - General Campaign 
  • $2.39 on 2024-12-04 - General Campaign 
  • $20.97 on 2024-12-11 - General Campaign 
  • $40.30 on 2024-05-22 - General Campaign 
  • $3.18 on 2024-09-05 - General Campaign 
  • $15.81 on 2024-08-28 - General Campaign 
  • $5.16 on 2024-08-21 - General Campaign 
  • $7.13 on 2024-08-14 - General Ca

## 2. Advanced Donation Filtering

Search for donations with multiple criteria and analyze patterns.

In [3]:
# Search for large donations from a specific date range
search_request = {
    "searchFields": [
        {
            "field": "Donation Amount",
            "operator": "GREATER_AND_EQUAL",
            "value": 500
        },
        {
            "field": "Donation Date",
            "operator": "GREATER_AND_EQUAL",
            "value": "2024-01-01"
        },
        {
            "field": "Donation Date",
            "operator": "LESS_AND_EQUAL",
            "value": "2024-12-31"
        }
    ],
    "outputFields": [
        "Donation ID",
        "Account ID",
        "Campaign Name",
        "Fund",
        "Donation Date"
    ],
    "pagination": {
        "currentPage": 0,
        "pageSize": 200
    }
}

try:
    large_donations = list(client.donations.search(search_request))
    
    print(f"Found {len(large_donations)} large donations (≥$500) in 2024:")
    
    # Analyze by campaign
    campaign_totals = {}
    payment_method_counts = {}
    
    for donation in large_donations:
        amount = donation.get('Donation Amount', 0)
        campaign = donation.get('Campaign Name', 'Unknown')
        
        # Track campaign totals
        campaign_totals[campaign] = campaign_totals.get(campaign, 0) + amount
            
    print("\n=== Campaign Performance ===")
    for campaign, total in sorted(campaign_totals.items(), key=lambda x: x[1], reverse=True):
        print(f"  {campaign}: ${total:,.2f}")
            
except Exception as e:
    print(f"Error searching large donations: {e}")

Found 36 large donations (≥$500) in 2024:

=== Campaign Performance ===
  Office Campaign: $0.00
  General Campaign: $0.00
  Donate for Democracy: $0.00
  2024 Spring Celebration: $0.00


## 3. Creating New Donations

Demonstrate how to create donations programmatically.

In [4]:
# Example of creating a new donation
# Note: This is for demonstration - you'll need valid account IDs

# ⚠️ WARNING: This function modifies the database - execution is disabled for safety

# ⚠️ WARNING: This function modifies the database - execution is disabled for safety

def create_sample_donation(account_id: int, amount: float, campaign: str = "General Fund"):
    """Create a sample donation for testing."""
    
    donation_data = {
        "amount": amount,
        "date": datetime.now().strftime("%Y-%m-%d"),
        "Account ID": account_id,
        "campaign": campaign,
        "fund": "General Fund",
        "source": "API SDK Example",
        "paymentMethod": "Check",
        "note": "Created via Python SDK for demonstration"
    }
    
    try:
        # SAFETY: Commented out to prevent database modification
        # SAFETY: Commented out to prevent database modification
        # # result = client.donations.create(donation_data)
        print(f"Created donation {result.get('id')} for ${amount}")
        return result
    except Exception as e:
        print(f"Error creating donation: {e}")
        return None

# Uncomment to test donation creation (requires valid account ID)
# sample_donation = create_sample_donation(account_id=123, amount=100.00)

print("Donation creation function defined. Update with valid account ID to test.")

Donation creation function defined. Update with valid account ID to test.


## 4. Donation Analysis and Reporting

Analyze donation patterns and create summary reports.

In [5]:
# Analyze donation trends over time
def analyze_donation_trends(days_back: int = 90):
    """Analyze donation trends over the specified period."""
    
    start_date = (datetime.now() - timedelta(days=days_back)).strftime("%Y-%m-%d")
    
    search_request = {
        "searchFields": [
            {
                "field": "Donation Date",
                "operator": "GREATER_AND_EQUAL",
                "value": start_date
            }
        ],
        "outputFields": [
            "Donation ID",
            "Donation Amount",
            "Donation Date",
            "Campaign Name",
            "Source",
            "Pledge Amount"
        ],
        "pagination": {
            "currentPage": 0,
            "pageSize": 200  # Get more data for analysis
        }
    }
    
    try:
        donations = list(client.donations.search(search_request))
        
        if not donations:
            print(f"No donations found in the last {days_back} days")
            return
        
        # Calculate key metrics
        total_amount = sum(float(d.get('Donation Amount', 0)) for d in donations)
        average_donation = total_amount / len(donations) if donations else 0
        largest_donation = max(float(d.get('Donation Amount', 0)) for d in donations)
        smallest_donation = min(float(d.get('Donation Amount', 0)) for d in donations)

        print(f"=== Donation Analysis (Last {days_back} Days) ===")
        print(f"Total Donations: {len(donations)}")
        print(f"Total Amount: ${total_amount:,.2f}")
        print(f"Average Donation: ${average_donation:.2f}")
        print(f"Largest Donation: ${largest_donation:,.2f}")
        print(f"Smallest Donation: ${smallest_donation:.2f}")
        
        # Donation size distribution
        size_buckets = {
            "$1-$25": 0,
            "$26-$100": 0,
            "$101-$500": 0,
            "$501-$1000": 0,
            "$1000+": 0
        }

        for donation in donations:
            amount = float(donation.get('Donation Amount', 0))
            if amount <= 25:
                size_buckets["$1-$25"] += 1
            elif amount <= 100:
                size_buckets["$26-$100"] += 1
            elif amount <= 500:
                size_buckets["$101-$500"] += 1
            elif amount <= 1000:
                size_buckets["$501-$1000"] += 1
            else:
                size_buckets["$1000+"] += 1
        
        print("\n=== Donation Size Distribution ===")
        for size_range, count in size_buckets.items():
            percentage = (count / len(donations)) * 100 if donations else 0
            print(f"  {size_range}: {count} donations ({percentage:.1f}%)")

        print("\n=== Donation Source Distribution ===")
        sources = {}
        for donation in donations:
            source = donation.get("Source", "Unknown")
            sources[source] = sources.get(source, 0) + 1

        for source, count in sorted(sources.items(), key=lambda x: x[1], reverse=True)[:5]:
            print(f"  {source}: {count} donations")

        print("\n=== Pledge Analysis ===")
        pledge_donations = [d for d in donations if d.get('Pledge Amount')]
        print(f"  Donations with pledges: {len(pledge_donations)}")
        
        if pledge_donations:
            pledge_amount = sum(float(d.get('Amount', 0)) for d in pledge_donations)
            print(f"  Total pledge donations: ${pledge_amount:,.2f}")

        
    except Exception as e:
        print(f"Error analyzing donation trends: {e}")

# Run the analysis
analyze_donation_trends(365)

=== Donation Analysis (Last 365 Days) ===
Total Donations: 929
Total Amount: $62,389.81
Average Donation: $67.16
Largest Donation: $2,483.75
Smallest Donation: $0.00

=== Donation Size Distribution ===
  $1-$25: 587 donations (63.2%)
  $26-$100: 240 donations (25.8%)
  $101-$500: 82 donations (8.8%)
  $501-$1000: 16 donations (1.7%)
  $1000+: 4 donations (0.4%)

=== Donation Source Distribution ===
  None: 777 donations
  Online (NEON PAY): 152 donations

=== Pledge Analysis ===
  Donations with pledges: 929
  Total pledge donations: $0.00


## 5. Working with Recurring Donations

Manage recurring donation schedules and analyze subscription patterns.

In [6]:
# Search for active recurring donations
def analyze_recurring_donations():
    """Analyze recurring donation patterns."""
    
    try:
        # Get recent recurring donations
        recurring_donations = list(client.recurring_donations.list(
            page_size=200,
            limit=100
        ))
        
        if not recurring_donations:
            print("No recurring donations found")
            return
        
        print(f"=== Recurring Donations Analysis ===")
        print(f"Total Recurring Donations: {len(recurring_donations)}")
        
        # Analyze by status
        status_counts = {}
        active_total = 0
        frequency_counts = {}
        
        for recurring in recurring_donations:
            status = recurring.get('status', 'Unknown')
            amount = recurring.get('amount', 0)
            frequency = recurring.get('frequency', 'Unknown')
            
            status_counts[status] = status_counts.get(status, 0) + 1
            frequency_counts[frequency] = frequency_counts.get(frequency, 0) + 1
            
            if status.lower() == 'active':
                active_total += amount
        
        print("\n=== Status Breakdown ===")
        for status, count in sorted(status_counts.items()):
            print(f"  {status}: {count}")
        
        print("\n=== Frequency Breakdown ===")
        for frequency, count in sorted(frequency_counts.items()):
            print(f"  {frequency}: {count}")
        
        print(f"\nTotal Monthly Recurring Revenue (approx): ${active_total:,.2f}")
        
    except Exception as e:
        print(f"Error analyzing recurring donations: {e}")

analyze_recurring_donations()

Error analyzing recurring donations: HTTP 222: {'recurringDonations': [{'id': 8, 'accountId': '1141', 'donorName': 'Ellen Terwilliger', 'amount': 75.0, 'frequency': '1 MONTH', 'nextDate': '2025-03-04'}, {'id': 15, 'accountId': '1174', 'donorName': 'Glenn Reynolds', 'amount': 75.0, 'frequency': '1 MONTH', 'nextDate': '2025-03-18'}, {'id': 10, 'accountId': '1248', 'donorName': 'Jill Christopherson', 'amount': 50.0, 'frequency': '1 MONTH', 'nextDate': '2025-03-06'}, {'id': 17, 'accountId': '1397', 'donorName': 'Lorraine Henning', 'amount': 75.0, 'frequency': '1 MONTH', 'nextDate': '2025-03-25'}, {'id': 1, 'accountId': '1511', 'donorName': 'Nick Smiar', 'amount': 100.0, 'frequency': '1 MONTH', 'nextDate': '2025-02-12'}, {'id': 5, 'accountId': '1511', 'donorName': 'Nick Smiar', 'amount': 100.0, 'frequency': '1 MONTH', 'nextDate': '2025-02-28'}, {'id': 2, 'accountId': '1741', 'donorName': 'Test2 test2', 'amount': 2.0, 'frequency': '1 MONTH', 'nextDate': '2025-02-04'}, {'id': 3, 'accountId': 

## 6. Donation Data Export

Export donation data for external analysis or reporting.

In [7]:
# Export donations to CSV format
import csv
from io import StringIO

def export_donations_to_csv(start_date: str, end_date: str, limit: int = 500):
    """Export donations within date range to CSV format."""
    
    search_request = {
        "searchFields": [
            {
                "field": "Donation Date",
                "operator": "GREATER_AND_EQUAL",
                "value": start_date
            },
            {
                "field": "Donation Date",
                "operator": "LESS_AND_EQUAL",
                "value": end_date
            }
        ],
        "outputFields": [
            "Donation ID",
            "Account ID",
            "Donation Amount",
            "Donation Date",
            "Campaign Name",
            "Fund",
            "Source",
            "Donor Note"
        ],
        "pagination": {
            "currentPage": 0,
            "pageSize": min(limit, 200)
        }
    }
    
    try:
        donations = list(client.donations.search(search_request))
        
        if not donations:
            print(f"No donations found between {start_date} and {end_date}")
            return
        
        # Create CSV in memory
        output = StringIO()
        
        # Get field names from first record
        fieldnames = list(donations[0].keys())
        writer = csv.DictWriter(output, fieldnames=fieldnames)
        
        writer.writeheader()
        writer.writerows(donations)
        
        csv_content = output.getvalue()
        output.close()
        
        # Display sample of CSV (first few lines)
        lines = csv_content.split('\n')[:6]  # Header + 5 data rows
        print(f"=== Sample CSV Export ({len(donations)} total records) ===")
        for line in lines:
            if line.strip():
                print(line)
        
        if len(donations) > 5:
            print("... (additional rows not shown)")
        
        # In a real application, you would save this to a file:
        # with open(f'donations_{start_date}_to_{end_date}.csv', 'w', newline='') as f:
        #     f.write(csv_content)
        
        return csv_content
        
    except Exception as e:
        print(f"Error exporting donations: {e}")
        return None

# Export donations from the last 30 days
start_date = (datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d")
end_date = datetime.now().strftime("%Y-%m-%d")

csv_data = export_donations_to_csv(start_date, end_date, limit=20)

=== Sample CSV Export (85 total records) ===
Account ID,Fund,Donation Amount,Donation ID,Donation Date,Source,Donor Note,Campaign Name
1737,RCU General Account,2.58,5084,2025-09-04,,,General Campaign
923,RCU General Account,10.00,5159,2025-09-19,,,General Campaign
923,RCU General Account,63.67,5166,2025-09-19,,,General Campaign
923,RCU General Account,63.66,5176,2025-09-19,,,General Campaign
923,RCU General Account,57.00,5088,2025-09-05,,,General Campaign
... (additional rows not shown)


## 7. Best Practices and Tips

### Performance Optimization
- Use pagination for large datasets
- Request only the fields you need in `outputFields`
- Use date ranges to limit search scope
- Cache field definitions when possible

### Data Quality
- Always validate donation amounts
- Check for duplicate donations
- Use proper date formatting (YYYY-MM-DD)
- Include meaningful campaign and fund information

### Error Handling
- Wrap API calls in try-catch blocks
- Check for required fields before creating donations
- Validate account IDs exist before creating donations
- Monitor rate limits for bulk operations